In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost/info/prophage_data.clusters_80.phageboost_70.2504.tsv \
/media/concha-eloko/Linux/PPT_clean

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Dpo_domains.0805.final.extra.multi.fasta \
/media/concha-eloko/Linux/PPT_clean

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Dpo_domains.0805.final.ultimate.multi.fasta \
/media/concha-eloko/Linux/PPT_clean

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/full_prediction.vsBeaDB.txt \
/media/concha-eloko/Linux/PPT_clean

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Minibatch_hits.75bits.tsv \
/media/concha-eloko/Linux/PPT_clean

# Goal : Generate the final Dataframe with the non redundant protein sequences, with all the relevant information
***
## I. DepoScope method (Anubis return)
## II. HMM profile DB (PPT)
## III. Blastp Bea DB (Minibatch)
***

### I. Deposcope method :

> Anubis return

In [67]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"

info_col = ["prot_name","start","end","domain_seq","full_seq","dpo_index"]

df_prophage_label = pd.read_csv(f"{path_project}/prophage_data.clusters_80.phageboost_70.2504.tsv" , sep = "\t", skiprows=1)
df_prophage_label.columns = ["Prophage_name","KL_type","Infected_ancestor","n_clades","siblings","n_ancestors","n_KL_swaps","old_KL_types","all_old_KL_types"]

df_anubis_return_emb = pd.read_csv(f"{path_project}/embeddings/anubis_return.esm2.embedding.csv" , sep = ",", header = None)
df_anubis_return_info = pd.read_csv(f"{path_project}/Anubis_return.predictions.0709.big.annotated.tsv" , sep = "\t", names = info_col)

In [75]:
df_anubis_return_info_150 = df_anubis_return_info[df_anubis_return_info["domain_seq"].str.len() >= 150]
df_anubis_return_info_150["Dataset"] = "anubis_return"
df_anubis_return_info_150_core = df_anubis_return_info_150[["dpo_index","Dataset","domain_seq", "full_seq"]]
df_anubis_return_info_150_core = df_anubis_return_info_150_core.drop_duplicates(subset = ["dpo_index"] , keep = "first")

# Write the file :
df_anubis_return_info_150_core.to_csv(f"{path_work}/DF_core.anubis_return_150.csv" , header = 0, index = False , sep = "\t")

In [76]:
df_anubis_return_info_150_core

,dpo_index,Dataset,domain_seq,full_seq
0,anubis_return__0,anubis_return,ALRTLQSRLKDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPG...,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...
1,anubis_return__1,anubis_return,AEPADDDDSPAFTGTGWVADDVGSYVRINSGLVLIKSITSAQVAVG...,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...
2,anubis_return__2,anubis_return,KDKNTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSD...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
4,anubis_return__3,anubis_return,GRLWKRRFEGNKIRMAWAGGKSQYQTSDPQDAAFKKCLKAAASLSA...,MRDLILQLSKSSIYSTKPFSARNQYIRYDNQYYRLNAATDVGFTTT...
5,anubis_return__4,anubis_return,GFALDTTGLMAIRPGAFDDLPNVNNCKNIFTNCSSLTGIPASLFSR...,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...
...,...,...,...,...
21296,anubis_return__4303,anubis_return,DGAKLVGGIGFVSPEMFGYLHGVSPDAGPYYQKAVDEGHARGLPVQ...,MADNEKLGSTSPQVLLRNATNLDKLVNDRESESLPDRFDVLRRTWF...
21297,anubis_return__4304,anubis_return,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...
21298,anubis_return__4305,anubis_return,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...
21299,anubis_return__4306,anubis_return,PEPKRMFNIKGKERIELTPAELATFNTSYSQYLKKGSNYLPYPKLY...,MAVLNPPLGTTTPEIFLDNVKRADELVNGPAGTVSDRAGEPLDTWR...


***
### II. PD domain DB :

In [80]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
from Bio import SeqIO

warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"

info_col = ["prot_name","start","end","domain_seq","full_seq","dpo_index"]

df_prophage_label = pd.read_csv(f"{path_project}/prophage_data.clusters_80.phageboost_70.2504.tsv" , sep = "\t", skiprows=1)
df_prophage_label.columns = ["Prophage_name","KL_type","Infected_ancestor","n_clades","siblings","n_ancestors","n_KL_swaps","old_KL_types","all_old_KL_types"]

In [92]:
# The domains dico : 
dico_ultimate_fasta = {record.id : record.seq for record in SeqIO.parse(f"{path_project}/Dpo_domains.0805.final.ultimate.multi.fasta", "fasta")}

# The info dataframe
DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )
DF_embeddings.rename(columns={0: 'index'}, inplace=True)

# The dpos within PPT
all_data = pd.merge(DF_info , DF_embeddings , on = "index")
all_data_info = all_data[["Phage", "KL_type_LCA", "Infected_ancestor", "index","Dataset", "seq"]]
all_data_info_ppt = all_data_info[all_data_info["Dataset"] == "ppt"]
all_data_info_ppt["domain_seq"] = all_data_info_ppt["index"].apply(lambda x : str(dico_ultimate_fasta[x]))

In [97]:
all_data_info_ppt_core = all_data_info_ppt.drop_duplicates(subset = ["seq"], keep = "first")
df_ppt_unique = all_data_info_ppt_core[["index","Dataset","domain_seq", "seq"]]
df_ppt_unique.to_csv(f"{path_work}/DF_core.ppt.csv" , header = 0, index = False , sep = "\t")

In [105]:
df_ppt_unique

,index,Dataset,domain_seq,seq
0,ppt__1,ppt,KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVS...,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...
2,ppt__4,ppt,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
6,ppt__21,ppt,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...
531,ppt__22,ppt,RIVRPEWYGCRGLGSSHPDTVPFAKMLSSLSDGDYVELRANAKYYN...,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...
2975,ppt__42,ppt,EIISPWDFNCKPDAVFDPVTQRLIDGTDNTANLQRMFSEAHYHGVD...,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...
...,...,...,...,...
15195,ppt__7145,ppt,VTPLLKPGVENAEYNYSILQAAADKNKQLSLPAGDFYISRAFEAGK...,MSFYLTGNPVPSASMPDVHDNAQNLDFALNEITSTFWTDRLGRRRM...
15196,ppt__7146,ppt,MFGYLHGVTPDAVPYFQAAVDEGHARGLPVQLTGKYYATTYPQKVT...,MFGYLHGVTPDAVPYFQAAVDEGHARGLPVQLTGKYYATTYPQKVT...
15197,ppt__7147,ppt,GFVSPEMFGYLQGVSPDAIPYYQAAVDEGHARGLPVKLTGKYYATT...,MPATPQDRLYGLTTSVAVKPPVYISADYDITRFGEQTITSKTPTDE...
15199,ppt__7149,ppt,GFVSPEMFGYLHGVSPDAGPYYQKAVDEGHARGLPVQLTGKYYATT...,MADNEKLGSTSPQVLLRNATNLDKLVNDRESESLPDRFDVLRRTWF...


***
### III. Minibatch : 

In [103]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"

info_col = ["prot_name","start","end","domain_seq","full_seq","dpo_index"]

df_prophage_label = pd.read_csv(f"{path_project}/prophage_data.clusters_80.phageboost_70.2504.tsv" , sep = "\t", skiprows=1)
df_prophage_label.columns = ["Prophage_name","KL_type","Infected_ancestor","n_clades","siblings","n_ancestors","n_KL_swaps","old_KL_types","all_old_KL_types"]

df_minibatch_candidates = pd.read_csv(f"{path_project}/Minibatch_hits.75bits.tsv", sep = "\t", names = ["protein_name", "bitscore", "seq"] )

In [104]:
df_minibatch_candidates

,protein_name,bitscore,seq
0,GCF_021535085.1__phage1__1337,710.0,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHVLVQTLDTSENILT...
1,GCF_015666635.1__phage8__77,479.0,MATRIINRNASLWASPKAKLSVPFAGPWEGFFSFGIDAATSVRNLI...
2,GCF_003262995.1__phage6__36,100.0,MAMYEVGTVTGAASQARVTGATTKWSQEALGILPGSILVVYRSGSA...
3,GCF_021137055.1__phage12__69,222.0,MSTYKTKNPLGSAAVKDLYDNAENVDKFVNDRTKEELEDRLGVLRK...
4,GCF_021137855.1__phage11__2056,89.0,MTQYNTGYPVPSPAMPDVWDNNETIDSFVNSPEISVTTRTGIVRDT...
...,...,...,...
16531,GCF_002185905.2__phage10__5,227.0,MVENDTSSVEYQLSTSTGPFSIPFYFIENGHIVAELYTQNGDDFNK...
16532,GCF_018108385.1__phage5__256,712.0,MANNIIKGRKGGGSKQRTPTEQPDDLQSVAKAKILLALGEGEFAGG...
16533,GCF_002157345.1__phage0__0,335.0,MDQFKKKRLPLLIAGAGGKKSSGSSRTPVEADDTVNSRAMASILDL...
16534,GCF_002278055.1__phage6__1550,100.0,MAMYEVGTVTGAASQARVTGATTKWSQEALGILPGSILVVYRSGSA...


In [112]:
df_anubis_return_info_150_core

,dpo_index,Dataset,domain_seq,full_seq
0,anubis_return__0,anubis_return,ALRTLQSRLKDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPG...,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...
1,anubis_return__1,anubis_return,AEPADDDDSPAFTGTGWVADDVGSYVRINSGLVLIKSITSAQVAVG...,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...
2,anubis_return__2,anubis_return,KDKNTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSD...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
4,anubis_return__3,anubis_return,GRLWKRRFEGNKIRMAWAGGKSQYQTSDPQDAAFKKCLKAAASLSA...,MRDLILQLSKSSIYSTKPFSARNQYIRYDNQYYRLNAATDVGFTTT...
5,anubis_return__4,anubis_return,GFALDTTGLMAIRPGAFDDLPNVNNCKNIFTNCSSLTGIPASLFSR...,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...
...,...,...,...,...
21296,anubis_return__4303,anubis_return,DGAKLVGGIGFVSPEMFGYLHGVSPDAGPYYQKAVDEGHARGLPVQ...,MADNEKLGSTSPQVLLRNATNLDKLVNDRESESLPDRFDVLRRTWF...
21297,anubis_return__4304,anubis_return,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...
21298,anubis_return__4305,anubis_return,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...
21299,anubis_return__4306,anubis_return,PEPKRMFNIKGKERIELTPAELATFNTSYSQYLKKGSNYLPYPKLY...,MAVLNPPLGTTTPEIFLDNVKRADELVNGPAGTVSDRAGEPLDTWR...


In [113]:
df_ppt_unique

,index,Dataset,domain_seq,seq
0,ppt__1,ppt,KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVS...,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...
2,ppt__4,ppt,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
6,ppt__21,ppt,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...
531,ppt__22,ppt,RIVRPEWYGCRGLGSSHPDTVPFAKMLSSLSDGDYVELRANAKYYN...,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...
2975,ppt__42,ppt,EIISPWDFNCKPDAVFDPVTQRLIDGTDNTANLQRMFSEAHYHGVD...,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...
...,...,...,...,...
15195,ppt__7145,ppt,VTPLLKPGVENAEYNYSILQAAADKNKQLSLPAGDFYISRAFEAGK...,MSFYLTGNPVPSASMPDVHDNAQNLDFALNEITSTFWTDRLGRRRM...
15196,ppt__7146,ppt,MFGYLHGVTPDAVPYFQAAVDEGHARGLPVQLTGKYYATTYPQKVT...,MFGYLHGVTPDAVPYFQAAVDEGHARGLPVQLTGKYYATTYPQKVT...
15197,ppt__7147,ppt,GFVSPEMFGYLQGVSPDAIPYYQAAVDEGHARGLPVKLTGKYYATT...,MPATPQDRLYGLTTSVAVKPPVYISADYDITRFGEQTITSKTPTDE...
15199,ppt__7149,ppt,GFVSPEMFGYLHGVSPDAGPYYQKAVDEGHARGLPVQLTGKYYATT...,MADNEKLGSTSPQVLLRNATNLDKLVNDRESESLPDRFDVLRRTWF...


In [120]:
df_minibatch_selection = df_minibatch_candidates[df_minibatch_candidates["seq"].isin(df_anubis_return_info_150_core["full_seq"].tolist() + df_ppt_unique["seq"].tolist())]
df_minibatch_selection_unique = df_minibatch_selection.drop_duplicates(subset = ["seq"], keep = "first")
df_minibatch_selection_unique.to_csv(f"{path_project}/DF_Minibatch_75bits.core.tsv", sep = "\t", header = 0, index = False)

***
# II. Make the dataframe :

> open the DF

In [4]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
from Bio import SeqIO

tqdm.pandas()
warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"
info_col = ["prot_name","start","end","domain_seq","full_seq","dpo_index"]

# info files :
df_prophage_label = pd.read_csv(f"{path_project}/prophage_data.clusters_80.phageboost_70.2504.tsv" , sep = "\t", skiprows=1)
df_prophage_label.columns = ["Prophage_name","KL_type","Infected_ancestor","n_clades","siblings","n_ancestors","n_KL_swaps","old_KL_types","all_old_KL_types"]

df_anubis_return_info = pd.read_csv(f"{path_project}/Anubis_return.predictions.0709.big.annotated.tsv" , sep = "\t", names = info_col)
df_minibatch_candidates = pd.read_csv(f"{path_project}/Minibatch_hits.75bits.tsv", sep = "\t", names = ["protein_name", "bitscore", "seq"])
df_OldSession_info =  pd.read_csv(f"{path_project}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
dico_ultimate_fasta = {record.id : record.seq for record in SeqIO.parse(f"{path_project}/Dpo_domains.0805.final.ultimate.multi.fasta", "fasta")}

# embeddings files : 
df_anubis_return_emb = pd.read_csv(f"{path_project}/embeddings/anubis_return.esm2.embedding.csv" , sep = ",", header = None)
df_anubis_return_emb = df_anubis_return_emb.drop([1281] , axis = 1)
df_OldSession_emb = pd.read_csv(f"{path_project}/embeddings/Dpo.2705.embeddings.ultimate.csv" , sep = ",", header = None)
df_OldSession_emb.rename(columns={0: 'index'}, inplace=True)

df_OldSession_info_plus = pd.merge(df_OldSession_info , df_OldSession_emb , on = "index")
df_OldSession_info_plus_info = df_OldSession_info_plus[["Phage", "KL_type_LCA", "Infected_ancestor", "index","Dataset", "seq"]]

# Core files : 
core_names = ["index", "dataset", "domain_seq", "full_seq"]
df_core_anubis_return = pd.read_csv(f"{path_project}/DF_core.anubis_return_150.csv" , names = core_names , sep = "\t")
df_core_ppt = pd.read_csv(f"{path_project}/DF_core.ppt.csv" , names = core_names , sep = "\t")
df_core_minibatch = pd.read_csv(f"{path_project}/DF_Minibatch_75bits.core.tsv", sep = "\t", names = ["protein_name","bitscore","full_seq"])
df_dom_classic = pd.read_csv(f"{path_project}/Dpo_domains.0805.final.ultimate.tsv", sep = "\t", names = ["0","index_dpo","fold","score","delineation","dom_seq","full_seq"])


***
### The PPT and Minibatch parts : 

In [7]:
# Filter out anubis and add domain seq
df_OldSession_info_plus_classic = df_OldSession_info_plus[df_OldSession_info_plus["Dataset"] != "anubis"]
df_OldSession_info_plus_classic["domain_seq"] = df_OldSession_info_plus_classic["index"].apply(lambda x : str(dico_ultimate_fasta[x]))

# Make the clean DFs
additional_columns = [int(i) for i in range(1, 1281)]
clean_df_classic_info = df_OldSession_info_plus_classic[["Phage","Protein_name","KL_type_LCA","Infected_ancestor","index","Dataset","seq","domain_seq"]]
clean_df_classic = pd.concat([clean_df_classic_info, df_OldSession_info_plus_classic[additional_columns]], axis=1)

#clean_df_classic.to_csv(f"{path_project}/DF_Dpo.final.1309.tsv", sep = "\t" ,  header = True, index = False)
#clean_df_classic_info.to_csv(f"{path_project}/DF_Dpo.final.info.1309.tsv", sep = "\t" ,  header = True, index = False)

***
### The Anubis part

In [9]:
# Get the old anubis part :
df_OldSession_info_plus_classic_anubis = df_OldSession_info_plus[df_OldSession_info_plus["Dataset"] == "anubis"]
df_OldSession_info_plus_classic_anubis

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
1,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__150,anubis,anubis__0,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,prophage_12186,0.036016,0.005938,...,0.008465,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658
4,GCF_004313505.1__phage4,KL14,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,-0.003019,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058
5,GCF_004313505.1__phage1,KL14,n830830,GCF_004313505.1__phage1__233,anubis,anubis__7,MAFNPELGSTSPAVLLDNAERLDKLVNGPAADVPDRGGDPLYSWRQ...,prophage_18,-0.028778,0.060945,...,-0.004240,-0.020966,0.073692,-0.047220,0.027809,0.073462,0.025343,-0.100966,0.013383,0.116881
971,GCF_902156295.1__phage5,KL107,n29132913,GCF_902156295.1__phage5__1396,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
972,GCF_000281355.1__phage23,KL107,81.9/1003170,GCF_000281355.1__phage23__13,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18261,GCF_015209405.1__phage17,KL2,81.9/1004099,GCF_015209405.1__phage17__0,anubis,anubis__1643,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,prophage_12,-0.014377,0.119705,...,0.040801,-0.001167,0.108840,0.008335,0.026747,0.046616,0.033648,-0.092224,0.048720,0.034757
18262,GCF_004311345.1__phage11,KL34,GCF_004311345.1,GCF_004311345.1__phage11__99,anubis,anubis__1644,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11944,-0.001691,-0.067717,...,0.119783,0.073706,0.057788,0.004341,0.007389,-0.081588,0.100995,-0.045545,0.021685,0.013167
18263,GCF_900172635.1__phage3,KL124,GCF_900172635.1,GCF_900172635.1__phage3__1811,anubis,anubis__1647,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11522,-0.002816,-0.055336,...,0.110956,0.056314,0.059700,0.004641,0.016676,-0.071711,0.095491,-0.055784,0.015615,0.000201
18264,GCF_013604525.1__phage18,KL30,n49324932,GCF_013604525.1__phage18__52,anubis,anubis__1649,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...,prophage_2576,-0.000031,0.046085,...,-0.016720,-0.006530,0.077754,-0.024675,0.026408,0.016113,0.043904,-0.091418,0.078778,0.105842


> Make the clean anubis_return DF

In [10]:
# Format the DF :
df_anubis_return_info.rename(columns={"dpo_index": 'index'}, inplace=True)
df_anubis_return_emb.rename(columns={0: 'index'}, inplace=True)

# Add the label information
tqdm.pandas()
df_anubis_return_full = pd.merge(df_anubis_return_info , df_anubis_return_emb , on = "index")
df_anubis_return_full["KL_type_LCA"] = df_anubis_return_full["prot_name"].progress_apply(lambda x : df_prophage_label[df_prophage_label["Prophage_name"] == f'{"__".join(x.split("__")[0:-1])}.fasta']["KL_type"].values[0])
df_anubis_return_full["Infected_ancestor"] = df_anubis_return_full["prot_name"].progress_apply(lambda x : df_prophage_label[df_prophage_label["Prophage_name"] == f'{"__".join(x.split("__")[0:-1])}.fasta']["Infected_ancestor"].values[0])



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21301/21301 [01:56<00:00, 182.10it/s]


In [ ]:
# Write the DFs : 


In [ ]:
# When the sequence is present in the old anubis, add the old embeddings, else keep the anubis return ones
# The idea here is that the old anubis domains were computed through esmfold-SWORD2-Foldseek methods which is more accurate
# The sequences that are exclusively in the anubis return are likely triple helix domains, that are not treatable with 

anubis_integrated_embeddings = []

anubis_full_dict = df_anubis_return_full.to_dict(orient='index')
for index, row in anubis_full_dict.items() :


    

In [1]:
df_anubis_return_full

NameError: name 'df_anubis_return_full' is not defined

In [212]:
df_OldSession_info_plus_classic

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,domain_seq
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339,KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVS...
2,GCF_016651625.1__phage12,KL43,GCF_016651625.1,GCF_016651625.1__phage12__59,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_924,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...
3,GCF_019928025.1__phage0,KL43,n1471,GCF_019928025.1__phage0__10,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_2929,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...
6,GCF_902156295.1__phage7,KL107,n29132913,GCF_902156295.1__phage7__1791,ppt,ppt__21,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,prophage_15,0.035789,0.004226,...,-0.078897,-0.061039,0.000776,-0.054214,0.027240,0.044547,-0.133716,-0.016950,0.110996,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...
7,GCF_000281355.1__phage7,KL107,81.9/1003170,GCF_000281355.1__phage7__19,ppt,ppt__21,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,prophage_15,0.035789,0.004226,...,-0.078897,-0.061039,0.000776,-0.054214,0.027240,0.044547,-0.133716,-0.016950,0.110996,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,GCF_013029545.1__phage3,KL9,GCF_013029545.1,GCF_013029545.1__phage3__0,minibatch,minibatch__2143,MANIEKLGSSSPEVLLKNATNLDQLVNGRESESLPDRFGVLRKTWH...,prophage_10780,0.013563,0.030933,...,-0.000664,0.100319,-0.005986,0.022693,0.041644,0.035039,-0.089788,0.046495,0.081841,VVDFDVTWFGAIGDSNTANSATNTVAIQRAIDAADSAGIASVTFPK...
15476,GCF_003954895.1__phage19,KL53,GCF_003954895.1,GCF_003954895.1__phage19__6,minibatch,minibatch__2154,MSTYKTGHPLGSAAVKDLFDNAENLDFALNSLTALIWTDRLGKVRP...,prophage_12850,0.033799,0.068283,...,-0.038315,0.070747,0.030770,0.036706,0.069955,0.028108,-0.122951,0.071033,0.075854,GKCPNISTLRTIEPSYSGQSIILEKCDAEGVLLNVQLHHDPYDIFS...
15478,GCF_900507555.1__phage4,KL62,n863,GCF_900507555.1__phage4__561,minibatch,minibatch__2164,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTLDTSESIVT...,prophage_5499,-0.030301,0.040220,...,-0.036106,0.093564,0.001933,-0.001696,0.118636,0.036329,-0.100558,0.028788,0.115129,SPRISDYKVTKGADATLAFAKAFAKAVETTGVVIVDVSCVLTTAVV...
15479,GCF_014946705.2__phage3,KL25,n1626,GCF_014946705.2__phage3__1251,minibatch,minibatch__2173,MKINEISQWEEEIYLLRRNDRVLGGVDGVANMQARQLANRTQYLKA...,prophage_3676,0.014216,0.026744,...,-0.034575,0.093013,-0.000460,0.031445,0.070793,0.030034,-0.104007,0.027653,0.084947,QATIFDFIKNETDIKTLKYAAGVDVDVSRAVEDAIKAGKTALYFPP...


***
## Write the final DF dico 

> The general info :

In [166]:
df_OldSession_info

,Phage,KL_type_LCA,Infected_ancestor,index,Dataset,seq
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,ppt__1,ppt,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...
1,GCF_016651625.1__phage29,KL43,GCF_016651625.1,anubis__0,anubis,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...
2,GCF_016651625.1__phage12,KL43,GCF_016651625.1,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
3,GCF_019928025.1__phage0,KL43,n1471,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...
4,GCF_004313505.1__phage4,KL14,GCF_004313505.1,anubis__5,anubis,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...
...,...,...,...,...,...,...
18261,GCF_015209405.1__phage17,KL2,81.9/1004099,anubis__1643,anubis,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...
18262,GCF_004311345.1__phage11,KL34,GCF_004311345.1,anubis__1644,anubis,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...
18263,GCF_900172635.1__phage3,KL124,GCF_900172635.1,anubis__1647,anubis,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...
18264,GCF_013604525.1__phage18,KL30,n49324932,anubis__1649,anubis,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...


In [170]:
# Write the ppt and minibatch parts :
dico_ultimate_fasta = {record.id : record.seq for record in SeqIO.parse(f"{path_project}/Dpo_domains.0805.final.ultimate.multi.fasta", "fasta")}

with open(f"{path_project}/DF_Dpo.final.1209.tsv", "w") as outfile :
    outfile.write(f"Phage\tProtein_name\tKL_type_LCA\tInfected_ancestor\tindex\tDataset\tfull_seq\tdomain_seq\n")
    for _, row in tqdm(df_OldSession_info_classic.iterrows()) :
        phage = row["Phage"]
        KL_type = df_prophage_label[df_prophage_label["Prophage_name"] == f"{phage}.fasta"]["KL_type"].values[0]
        infected_ancestor = df_prophage_label[df_prophage_label["Prophage_name"] == f"{phage}.fasta"]["Infected_ancestor"].values[0]
        id_dpo = row["index"]
        db_id = row["Dataset"]
        try : 
            dom_seq = dico_ultimate_fasta["id_dpo"]
            outfile.write(f"{phage}\t{row['Protein_name']}\t{KL_type}\t{infected_ancestor}\t{id_dpo}\t{db_id}\t{row['seq']}\t{dom_seq}\n")
        except Exception as e : 
            print(e , id_dpo)


20it [00:00, 95.71it/s]

'id_dpo' ppt__1
'id_dpo' ppt__4
'id_dpo' ppt__4
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21


43it [00:00, 104.93it/s]

'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21
'id_dpo' ppt__21


65it [00:00, 106.80it/s]

...

2217it [00:19, 106.46it/s]

'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170


2236it [00:20, 111.04it/s]


'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' ppt__170
'id_dpo' minibatch__773


KeyboardInterrupt: 

In [185]:
dico_ultimate_fasta["ppt__1"]

Seq('KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVSLISGARLI...RPR')

In [146]:
df_OldSession_info_classic = df_OldSession_info[df_OldSession_info["Dataset"] != "anubis"]
df_anubis_return_info_working = df_anubis_return_info[~df_anubis_return_info["full_seq"].isin(df_OldSession_info_classic["seq"].tolist())]
df_anubis_return_info_working

,prot_name,start,end,domain_seq,full_seq,dpo_index
1,GCF_016651625.1__phage29__150,188,492,AEPADDDDSPAFTGTGWVADDVGSYVRINSGLVLIKSITSAQVAVG...,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,anubis_return__1
4,GCF_021460965.1__phage15__4550,136,537,GRLWKRRFEGNKIRMAWAGGKSQYQTSDPQDAAFKKCLKAAASLSA...,MRDLILQLSKSSIYSTKPFSARNQYIRYDNQYYRLNAATDVGFTTT...,anubis_return__3
5,GCF_019336255.1__phage7__230,367,1009,GFALDTTGLMAIRPGAFDDLPNVNNCKNIFTNCSSLTGIPASLFSR...,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,anubis_return__4
6,GCF_016925255.1__phage4__3029,367,1009,GFALDTTGLMAIRPGAFDDLPNVNNCKNIFTNCSSLTGIPASLFSR...,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,anubis_return__4
7,GCF_019683915.1__phage23__530,367,1009,GFALDTTGLMAIRPGAFDDLPNVNNCKNIFTNCSSLTGIPASLFSR...,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,anubis_return__4
...,...,...,...,...,...,...
21293,GCF_013732655.1__phage2__1307,413,712,GVAAGDWETSSGSVQVSLEYSGGSLTGTVRITAVNSRTSATGIVLS...,MPRNNVPLLAFNRGIISPLALARTDIERLALSAEVQTNWMPRLLGS...,anubis_return__4300
21294,GCF_013732655.1__phage5__1995,219,328,GNLNMGGQSVYNAQNITANGTTTTGILKTNAAATVGTTLNVAGVTT...,MKMKIKKGFSLLELTLVLGVGTIVAFMKFQDMKNEQESILASAVGQ...,anubis_return__4301
21297,GCF_018137815.1__phage9__1,0,422,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,anubis_return__4304
21299,GCF_016903195.1__phage0__1045,390,837,PEPKRMFNIKGKERIELTPAELATFNTSYSQYLKKGSNYLPYPKLY...,MAVLNPPLGTTTPEIFLDNVKRADELVNGPAGTVSDRAGEPLDTWR...,anubis_return__4306


In [147]:
df_anubis_return_info

,prot_name,start,end,domain_seq,full_seq,dpo_index
0,GCF_016651625.1__phage29__142,226,664,ALRTLQSRLKDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPG...,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,anubis_return__0
1,GCF_016651625.1__phage29__150,188,492,AEPADDDDSPAFTGTGWVADDVGSYVRINSGLVLIKSITSAQVAVG...,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,anubis_return__1
2,GCF_016651625.1__phage12__59,37,401,KDKNTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSD...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,anubis_return__2
3,GCF_019928025.1__phage0__10,37,401,KDKNTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSD...,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,anubis_return__2
4,GCF_021460965.1__phage15__4550,136,537,GRLWKRRFEGNKIRMAWAGGKSQYQTSDPQDAAFKKCLKAAASLSA...,MRDLILQLSKSSIYSTKPFSARNQYIRYDNQYYRLNAATDVGFTTT...,anubis_return__3
...,...,...,...,...,...,...
21296,GCF_013732655.1__phage3__1529,142,615,DGAKLVGGIGFVSPEMFGYLHGVSPDAGPYYQKAVDEGHARGLPVQ...,MADNEKLGSTSPQVLLRNATNLDKLVNDRESESLPDRFDVLRRTWF...,anubis_return__4303
21297,GCF_018137815.1__phage9__1,0,422,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,MSVYQSWRTAWRTPGVLRNYPPNPLYVDTVRGKASGKGSVDDPVNS...,anubis_return__4304
21298,GCF_018137815.1__phage7__6,0,417,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...,MLGGFIVFIAFMNFAQGGDIRNVSQPEIPLSCKVLKADKSVDSTRA...,anubis_return__4305
21299,GCF_016903195.1__phage0__1045,390,837,PEPKRMFNIKGKERIELTPAELATFNTSYSQYLKKGSNYLPYPKLY...,MAVLNPPLGTTTPEIFLDNVKRADELVNGPAGTVSDRAGEPLDTWR...,anubis_return__4306


In [ ]:
# Write the anubis_return part :
with open(f"{path_project}/DF_Dpo.final.12090.tsv", "a+") as outfile :
    for _, row in tqdm(df_OldSession_info.iterrows()) :
        phage = row["Phage"]
        KL_type = df_prophage_label[df_prophage_label["Prophage_name"] == f"{phage}.fasta"]["KL_type"].values[0]
        infected_ancestor = df_prophage_label[df_prophage_label["Prophage_name"] == f"{phage}.fasta"]["Infected_ancestor"].values[0]
        id_dpo = row["index"]
        db_id = row["Dataset"]
        outfile.write(f"{phage}\t{KL_type}\t{infected_ancestor}\t{row['Protein_name']}\t{db_id}\t{id_dpo}\t{row['seq']}\n")

In [ ]:
with open(f"{path_project}/DF_Dpo.final.1005.tsv", "w") as outfile :
    outfile.write(f"Phage\tKL_type_LCA\tInfected_ancestor\tProtein_name\tDataset\tindex\tseq\n")
    for _,row in tqdm(final_df.iterrows()) :
        phage = "__".join(row["protein_name"].split("__")[0:-1])
        KL_type = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["KL_type"].values[0]
        infected_ancestor = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["Infected_ancestor"].values[0]
        db_id = f"ppt__{row['index_seq']}"
        outfile.write(f"{phage}\t{KL_type}\t{infected_ancestor}\t{row['protein_name']}\tppt\t{db_id}\t{row['sequence']}\n")

> See the differences :

In [ ]:
dico_similaire = {"ppt" : [] , "minibatch" : [], "anubis" : [], "new" : []}
for seq in df_anubis_return_info_150["full_seq"].unique() : 
    if seq in all_data_info_unique["seq"].tolist() : 
        origin = all_data_info_unique[all_data_info_unique["seq"] == seq]["Dataset"].values[0]
        dico_similaire[origin].append(seq)
    else : 
        dico_similaire["new"].append(seq)